In [1]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import pandas as pd
import os
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import LeaveOneOut
from imblearn.over_sampling import SMOTE, ADASYN
#from imblearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.impute import SimpleImputer

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, LSTM, Embedding
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing import sequence

## preparing data

In [3]:
file_name = "fixations_training_data.xlsx"

In [4]:
data_sheet = "fixation_data"
lbl_sheet = "demographic"

In [5]:
df1 = pd.read_excel(file_name, sheet_name= data_sheet)
df2 = pd.read_excel(file_name, sheet_name=lbl_sheet)


In [6]:
df1['sample'] = "sample"

In [7]:
df1['sample'] = df1['sample'].str.cat([ df1['Subject'].astype(str), df1['Trial'].astype(str)], sep=' ')

In [8]:
df1['D'] = df1['AOI_Group'] == 'D'
df1['N'] = df1['AOI_Group'] == 'N'
df1['WS'] = df1['AOI_Group'] == 'White Space'


In [9]:
train_data = []
train_targets = []
dropping_list = ['Position_X', 'Position_Y', 'Number',  'Trial', 'Stimulus', 'Area_of_Interest', 'AOI_Group']
sbj = ['Subject','sample']

In [10]:
df1 = df1.drop(dropping_list, 1)
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
df1[['Fixation_Duration','Average_Pupil_Diameter']] = imp_mean.fit_transform(df1[['Fixation_Duration','Average_Pupil_Diameter']])

In [11]:
for code in df1['sample'].unique():
    tmp_df = df1[df1['sample'] == code]
    current_sub = tmp_df["Subject"].values[0]
    target = df2[df2['Subject'] == current_sub]['LSAS_Total'] > 50
    train_data.append(tmp_df.drop(sbj, 1).values)
    try:
        train_targets.append(target.values[0])
    except IndexError:
        train_targets.append(False)


In [12]:
X = np.array(train_data)

## LSTM

In [74]:
hidden_size = 40
y = train_targets
maxlen = 23

In [75]:
X = sequence.pad_sequences(X, maxlen=maxlen)

In [76]:
X_train, X_test, y_train, y_test = train_test_split(X, y)


In [77]:
model = Sequential()
#model.add(Embedding(5, 128))
model.add(LSTM(hidden_size, input_shape=(23, 5), dropout = 0.3, recurrent_dropout=0.3))
#model.add(Dense(10))
#model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Dropout(0.5))
model.add(Dense(1))
model.compile(optimizer='adam', loss='binary_crossentropy')
model.fit(np.array(X_train), np.array(y_train), epochs = 100)

Epoch 1/300
3871/3871 [==============================] - 12s 3ms/step - loss: 5.0200
Epoch 2/300
3871/3871 [==============================] - 4s 1ms/step - loss: 3.7237
Epoch 3/300
3871/3871 [==============================] - 4s 1ms/step - loss: 3.3551
Epoch 4/300
3871/3871 [==============================] - 4s 1ms/step - loss: 3.0489
Epoch 5/300
3871/3871 [==============================] - 4s 970us/step - loss: 2.9808
Epoch 6/300
3871/3871 [==============================] - 4s 1ms/step - loss: 1.8406
Epoch 7/300
3871/3871 [==============================] - 4s 1ms/step - loss: 1.5697
Epoch 8/300
3871/3871 [==============================] - 4s 1ms/step - loss: 1.0680A: 0s - 
Epoch 9/300
3871/3871 [==============================] - 5s 1ms/step - loss: 0.8280A: 1s
Epoch 10/300
3871/3871 [==============================] - 4s 1ms/step - loss: 0.8236
Epoch 11/300
3871/3871 [==============================] - 5s 1ms/step - loss: 0.8002
Epoch 12/300
3871/3871 [==============================] - 

3871/3871 [==============================] - 4s 1ms/step - loss: 0.6864
Epoch 96/300
3871/3871 [==============================] - 5s 1ms/step - loss: 0.6865
Epoch 97/300
3871/3871 [==============================] - 4s 1ms/step - loss: 0.6864
Epoch 98/300
3871/3871 [==============================] - 4s 1ms/step - loss: 0.6866
Epoch 99/300
3871/3871 [==============================] - 5s 1ms/step - loss: 0.6864
Epoch 100/300
3871/3871 [==============================] - 5s 1ms/step - loss: 0.6865A:
Epoch 101/300
3871/3871 [==============================] - 4s 1ms/step - loss: 0.6861A: 1s - loss: 0.6 -
Epoch 102/300
3871/3871 [==============================] - 4s 1ms/step - loss: 0.6866A: 0s - loss: 0.6
Epoch 103/300
3871/3871 [==============================] - 4s 1ms/step - loss: 0.6865
Epoch 104/300
3871/3871 [==============================] - 4s 1ms/step - loss: 0.6864
Epoch 105/300
3871/3871 [==============================] - 5s 1ms/step - loss: 0.6863
Epoch 106/300
3871/3871 [=========

3871/3871 [==============================] - 3s 886us/step - loss: 0.6862
Epoch 188/300
3871/3871 [==============================] - 4s 922us/step - loss: 0.6859
Epoch 189/300
3871/3871 [==============================] - 4s 986us/step - loss: 0.6865
Epoch 190/300
3871/3871 [==============================] - 3s 889us/step - loss: 0.6864
Epoch 191/300
3871/3871 [==============================] - 3s 882us/step - loss: 0.6866
Epoch 192/300
3871/3871 [==============================] - 3s 884us/step - loss: 0.6861
Epoch 193/300
3871/3871 [==============================] - 3s 897us/step - loss: 0.6865
Epoch 194/300
3871/3871 [==============================] - 3s 895us/step - loss: 0.6863
Epoch 195/300
3871/3871 [==============================] - 3s 880us/step - loss: 0.6864
Epoch 196/300
3871/3871 [==============================] - 3s 881us/step - loss: 0.6869
Epoch 197/300
3871/3871 [==============================] - 4s 913us/step - loss: 0.6865
Epoch 198/300
3871/3871 [=====================

3871/3871 [==============================] - 4s 969us/step - loss: 0.6864
Epoch 280/300
3871/3871 [==============================] - 3s 870us/step - loss: 0.6855
Epoch 281/300
3871/3871 [==============================] - 3s 823us/step - loss: 0.6862
Epoch 282/300
3871/3871 [==============================] - 3s 828us/step - loss: 0.6856
Epoch 283/300
3871/3871 [==============================] - 3s 894us/step - loss: 0.6863
Epoch 284/300
3871/3871 [==============================] - 3s 903us/step - loss: 0.6862
Epoch 285/300
3871/3871 [==============================] - ETA: 0s - loss: 0.6860- E - 3s 818us/step - loss: 0.6860
Epoch 286/300
3871/3871 [==============================] - 4s 938us/step - loss: 0.6862
Epoch 287/300
3871/3871 [==============================] - 3s 843us/step - loss: 0.6864
Epoch 288/300
3871/3871 [==============================] - 3s 845us/step - loss: 0.6865
Epoch 289/300
3871/3871 [==============================] - 3s 850us/step - loss: 0.6859
Epoch 290/300
3871

In [78]:
y_pred = model.predict(X_test) > 0.5
accuracy_score(y_test, y_pred)

0.5608055770720372

In [79]:
len(y_pred)

1291

In [80]:
sum(y_pred)

array([1291])